# Sobre

Este notebook contém os tópicos para a avaliação da segunda fase do processo seletivo do GPAM, 2/2019. A avaliação consiste na criação de um modelo para a classificação de discurso de ódio utilizando texto, usufruindo do dataset disponibilizado [no kaggle](https://www.kaggle.com/arkhoshghalb/twitter-sentiment-analysis-hatred-speech#train.csv).

Autor: Roger Lenke  
Matrícula: 15/0021437  
Email: rogerlenke@gmail.com  

Fontes utilizadas:  
[Machine Learning - Text Processing](https://towardsdatascience.com/machine-learning-text-processing-1d5a2d638958)  
[Working with Text Data - Sklearn](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  
[Natural Language Processing(NLP) for Machine Learning](https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b)  
[Feature Engineering for Twitter-based Applications](https://www.researchgate.net/publication/320550509_Feature_Engineering_for_Twitter-based_Applications)

# Análise Exploratória dos Dados

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

try:
  import nltk
except:
  !pip install nltk
  import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
# Loading the train dataset, which will be the only one used in this competition.

train_df = pd.read_csv('train.csv', encoding='utf8')

train_df.head()

## Análise Geral

In [0]:
train_df.shape

O dataset consiste em 31962 tweets, cada um classificado com a label  para tweets usuais e 1 para tweets com discurso de ódio. De acordo com a página do dataset, um tweet é classificado como discurso de ódio caso contenha racismo e/ou sexismo. 

In [0]:
# Check for missing data.
train_df['tweet'].isna().sum()

In [0]:
num_normal_tweets = train_df[train_df['label'] == 0].shape[0]
num_hate_tweets = train_df[train_df['label'] == 1].shape[0]

plt.pie([num_normal_tweets, num_hate_tweets], explode=[0.0, 0.2],
        labels=['Normal tweets', 'Hate speech tweets'],
        autopct='%1.1f%%');

In [0]:
print(f'Num normal tweets: {num_normal_tweets}')
print(f'Num hate tweets: {num_hate_tweets}')

O dataset está extremamente desbalanceado, com mais de 93% dos exemplos sendo de tweets usuais, e apenas 7% de tweets com discurso de ódio.

## Frequência de Palavras

In [0]:
# Concatenate all tweets in a string separating each tweet with a space.
tweets = train_df['tweet'].str.cat(sep=' ')

# Tokenize each word
tokens = word_tokenize(tweets)

def plot_words(tokens, amount, top=True):
  frequency = nltk.FreqDist(tokens)
  top_words = sorted(frequency, key=frequency.__getitem__, reverse=top)[:amount]
  top_words_counts = sorted(list(frequency.values()), reverse=top)[:amount]
  plt.plot(top_words, top_words_counts)
  plt.xticks(top_words, rotation='vertical');

plt.show()
plt.subplot(311)
plot_words(tokens, 25)
plt.subplot(313)
plot_words(tokens, 25, False)

Existem alguns caracteres de marcação comuns ao twitter (#, @, user) dentre os 25 tokens mais utilizados no corpo de texto, além de sinais de pontuação e stop-words. Algumas das palavras menos utilizadas contém caracteres ilógicos e números.

## Hashtags e Menções

Segundo o artigo *Feature Engineering for Twitter-based Applications*, as aplicações que utilizam dados do Twitter para realizar previsões podem se aproveitar de aspectos do texto e meta-dados, como as *hashtags* e as menções a outros usuários. Infelizmente, o dataset de treino não fornece metadados sobre os tweets presentes. Apesar disto, é possível retirar informações de *hashtags* e menções do texto disponível.

In [0]:
import re

expression = '#\w+'
train_df['hashtags'] = train_df['tweet'].apply(lambda tweet: re.findall(expression, tweet))

In [0]:
bins = np.linspace(0, 20, 20)

train_df['num_hashtags'] = train_df['hashtags'].apply(lambda hashtags: len(hashtags))

plt.hist(train_df[train_df['label'] == 0]['num_hashtags'], bins, alpha=0.5, label='normal');
plt.hist(train_df[train_df['label'] == 1]['num_hashtags'], bins, alpha=0.5, label='hate');
plt.legend(loc='upper left')

In [0]:
def flatten(l):
  return [item for sublist in l for item in sublist]

hashtags_normal = flatten(train_df[train_df['label'] == 0]['hashtags'].values)
hashtags_hate = flatten(train_df[train_df['label'] == 1]['hashtags'].values)

In [0]:
from collections import Counter

def most_common_hashtags(hashtags, amount):
  pair = [(hashtag, hashtag_count) for hashtag, hashtag_count in Counter(hashtags).most_common(amount)]
  hashtags = [hashtag for (hashtag, _) in pair]
  count = [count for (_, count) in pair]
  return hashtags, count

common_normal_hashtags, count_normal_hashtags = most_common_hashtags(hashtags_normal, 20)
common_hate_hashtags, count_hate_hashtags = most_common_hashtags(hashtags_hate, 20)

def plot_hashtags(labels, count):
  index = np.arange(0, len(labels))
  plt.bar(index, count)
  plt.xticks(index, labels, rotation='vertical');

plt.subplot(311)
plot_hashtags(common_normal_hashtags, count_normal_hashtags)
plt.subplot(313)
plot_hashtags(common_hate_hashtags, count_hate_hashtags)

O histograma demonstra que não há diferença na quantidade média de hashtags utilizadas nos tweets com discurso de ódio e tweets usuais. Porém, as hashtags mais utilizadas nos tweets de ódio são diferentes das hashtags mais utilizadas nos tweets usuais. Isto pode indicar que a feature hashtags é um bom diferenciador entre tweets com e sem discurso de ódio.

In [0]:
expression = '@user'
train_df['mentions'] = train_df['tweet'].apply(lambda tweet: len(re.findall(expression, tweet)))

In [0]:
bins = np.linspace(0, 20, 20)

plt.hist(train_df[train_df['label'] == 0]['mentions'], bins, alpha=0.5, label='normal');
plt.hist(train_df[train_df['label'] == 1]['mentions'], bins, alpha=0.5, label='hate');
plt.legend(loc='upper left');

O histograma denota que o número de mentions não é relevante para a separação entre os tweets com e sem discurso de ódio.

# Pré-processamento

A atividade de pré-processamento inclui a tokenização dos termos do texto, a limpeza de *stopwords*, de caracteres aleatórios, e a redução dos termos a raiz.

In [0]:
import string

def clean(tweet):
  tweet = word_tokenize(tweet)

  # Remove stop words
  stop_words = set(stopwords.words('english'))
  tweet = [x for x in tweet if not x in stop_words]

  # Remove Words with numbers
  tweet = [x for x in tweet if x.isalpha()]

  # Remove non-words
  tweet = [x for x in tweet if len(x) > 2]

  # Lemmanitize
  lemmatizer = WordNetLemmatizer() 
  tweet = [lemmatizer.lemmatize(x) for x in tweet]

  # Remove punctuation
  tweet = [x for x in tweet if x not in string.punctuation]

  return tweet

# Escolha das Features

O processo de extração das features envolve a conversão dos tokens de cada tweet para um formato que possa ser utilizados por modelos de aprendizado de máquina. Aqui, é utilizado o processo de word embedding, no qual palavras são representadas num espaço multidimensional, de maneira que palavras de sentido similar ficam mais próximas que palavras não relacionadas.

Além disto, o processo também envolve a adição de features extras na matriz gerada. Aqui, as hashtags utilizadas em cada tweet são adicionadas a matriz, a fim de atingir uma melhor capacidade de previsão.

Pelo fato dos dados de treinamento estarem bastante desbalanceados, optou-se por realizar um *undersampling* dos dados, especificamente os dados de tweets normais. Assim, espera-se evitar a situação de overfitting.

In [0]:
def vectorize(dataframe):
  tf_vectorizer = TfidfVectorizer(analyzer=clean)
  X_train_idf = tf_vectorizer.fit_transform(dataframe)
  return X_train_idf

X_train_idf = vectorize(train_df['tweet'])
X_train_idf.shape

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
import scipy as sp

def add_columns(matrix, dataframe):
  mlb = MultiLabelBinarizer(sparse_output=True)
  matrix = sp.sparse.hstack((matrix, mlb.fit_transform(dataframe)))
  return matrix

X_train_idf = add_columns(X_train_idf, train_df['hashtags'])

# Criação do modelo

Para o desafio, optou-se por utilizar o modelo SVM, ou *Support Vector Machines*.

O modelo SVM procura encontrar um hiperplano num espaço multidimensional onde estejam distribuídos os dados, de maneira que este hiperplano seja o que melhor separe as diferentes classes destes dados.

Existem diversos hiperplanos que separam os dados num espaço n-dimensional. O hiperplano escolhido pelo algoritmo SVM é aquele que maximiza a distância dentre os pontos de dados de cada classe que estão mais próximos.

In [0]:
from sklearn.svm import SVC
svm = SVC(probability=True, kernel='linear')

# Treinamento

Nesta etapa, foi realizada a divisão do dataset de treinamento entre dataset de treino e teste, de maneira a permitir uma avaliação da generalização do modelo criado em dados os quais não foram utilizados para treinamento.



In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_idf, train_df['label'], test_size=0.2, random_state=666)

svm.fit(X_train, y_train)

# Escolha da métrica

Devido ao desbalanceamento entre a quantidade de amostras da classe de tweets com discurso de ódio e classe de tweets sem discurso de ódio, a métrica de acurácia não é confiável para medir a qualidade do modelo, já que considera apenas a quantidade de previsões corretas em relação ao total de previsões.

As métricas de precisão e recall consideram, ambas, o **custo** de se realizar uma predição, de maneira que a precisão é utilizada quando a preocupação é evitar falsos positivos, enquanto o recall é a métrica utilizada quando o objetivo é selecionar o modelo que melhor evita falsos negativos.

Considerando que o modelo construído poderia ser utilizado para moderar os tweets realizados por usuários do twitter, é importante considerar tanto o custo de apagar um tweet que não contém discurso de ódio (falso positivo) quando o custo de não apagar um tweet que contém discurso de ódio (falso negativo). Por isto, a métrica **F1-Score**, que considera ambas a precisão e o recall, foi a escolhida para avaliar o modelo.

In [0]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix

y_pred = svm.predict(X_train)
print(f'F1 Score for training: {f1_score(y_train, y_pred)}')

y_pred = svm.predict(X_test)
print(f'F1 Score for test: {f1_score(y_test, y_pred)}')

# Avaliação

Para a visualização do score de acordo com a métrica F1, é obtida a probabilidade de um valor pertencer ou não a classe de tweets com discursos de ódio, para todos os tweets. Após isto, é plotado um gráfico variando o f1-score de acordo com diferentes *thresholds*, i.e, diferentes porcentagens que determinam se um dado será ou não predito como tweet com discurso de ódio.

In [0]:
proba = svm.predict_proba(X_test)
proba = proba[:, 1]
thresholds = np.arange(0.1, 0.9, 0.1)

scores = [f1_score(y_test, proba >= x) for x in thresholds]

plt.plot(thresholds, scores);
plt.title('F1-score with different thresholds');
plt.ylabel("F1-Score");
plt.xlabel("Threshold");
plt.show();

O gráfico demonstra que o score mais alto é obtido quando o threshold é fixado em 0.2, o que significa que qualquer tweet com 20% ou mais de probabilidade de conter discurso de ódio é classificado como tal.